In [1]:
import pandas as pd

In [2]:
from atomsci.ddm.pipeline import compare_models as cm
perf_df = cm.get_filesystem_perf_results('models/', pred_type='regression')
top_model=perf_df.sort_values(by="best_valid_r2_score", ascending=False).iloc[0]

Skipped loading some Jax models, missing a dependency. No module named 'haiku'
c:\Desktop\Hackathon\atomsci-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found data for 12 models under models/


In [4]:
from atomsci.ddm.pipeline import predict_from_model as pfm

best_split_data_file = "CDK_1_all_curated_" + top_model["split_strategy"] + "_" + top_model["splitter"] + "_" + top_model["split_uuid"] + ".csv"
best_split_data = pd.read_csv("Single_protein/" + best_split_data_file)


In [7]:
curated_data = pd.read_csv("Single_protein\scaled_descriptors\CDK_1_all_curated_with_rdkit_raw_descriptors.csv")

test_ids=best_split_data[best_split_data.subset == 'test'].cmpd_id.unique()
test_data = curated_data[curated_data.compound_id.isin(test_ids)]

# show most useful columns
test_data[['compound_id', 'base_rdkit_smiles', 'avg_pIC50']].head()

,compound_id,base_rdkit_smiles,avg_pIC50
0,CHEMBL2105759,CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)c...,4.522879
11,CHEMBL511478,NNC(=O)c1c(N)ncnc1Nc1ccc(OCc2ccccc2)c(Cl)c1,4.000000
13,CHEMBL402548,CO[C@@H](C(=O)N1Cc2[nH]nc(NC(=O)c3ccc(N4CCN(C)...,4.522879
16,CHEMBL3648106,CC(CO)Sc1nc(Nc2ccc(S(C)(=N)=O)cc2)ncc1Br,7.744727
35,CHEMBL3654248,C[C@@H]1CN(c2ccc(Nc3ncc4cc(C(=O)N(C)C)n(C5CCCC...,4.823909


In [12]:
model_dir = 'dataset/SLC6A3_models/SLC6A3_Ki_curated_model_9ff5a924-ef49-407c-a4d4-868a1288a67e.tar.gz'
input_df = test_data
id_col = 'compound_id'
smiles_col = 'base_rdkit_smiles'
response_col = 'avg_pIC50'

# loads a pretrained model from a model tarball file and runs predictions on
# compounds in an input data frame
pred_df = pfm.predict_from_model_file(model_path = top_model.model_path,
                                      input_df = test_data,
                                      id_col = id_col ,
                                      smiles_col = smiles_col,
                                      response_col = response_col,
                                      is_featurized=True)


# show most useful columns
pred_df[['compound_id', 'base_rdkit_smiles', 'avg_pIC50_actual','avg_pIC50_pred', 'avg_pIC50_std']].head()

Standardizing SMILES strings for 371 compounds.


C:\Desktop\Hackathon\AMPL\atomsci\ddm\pipeline\predict_from_model.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['orig_smiles']=input_df[smiles_col]
C:\Desktop\Hackathon\AMPL\atomsci\ddm\pipeline\predict_from_model.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[smiles_col] = std_smiles
2024-07-19 10:54:11,416 models/CDK_1_all_curated_model_96d385b9-4450-4a5a-8735-bc9324e3afbf.tar.gz, 1.6.1
2024-07-19 10:54:11,417 Version compatible check: models/CDK_1_all_curated_model_96d385

,compound_id,base_rdkit_smiles,avg_pIC50_actual,avg_pIC50_pred,avg_pIC50_std
0,CHEMBL2105759,CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)c...,4.522879,6.045669,1.339658
1,CHEMBL511478,NNC(=O)c1c(N)ncnc1Nc1ccc(OCc2ccccc2)c(Cl)c1,4.000000,5.158424,0.904387
2,CHEMBL402548,CO[C@@H](C(=O)N1Cc2[nH]nc(NC(=O)c3ccc(N4CCN(C)...,4.522879,5.438004,0.553633
3,CHEMBL3648106,CC(CO)Sc1nc(Nc2ccc(S(C)(=N)=O)cc2)ncc1Br,7.744727,7.014913,0.952868
4,CHEMBL3654248,C[C@@H]1CN(c2ccc(Nc3ncc4cc(C(=O)N(C)C)n(C5CCCC...,4.823909,4.837519,0.211383
